In [1]:
import csv
import exrex # !pip install exrex
import pandas as pd
import numpy as np
import re 
import spacy

from tqdm import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

# ebablbe auto-completion
%config Completer.use_jedi = False

## Generate Config File

download model.

In [2]:
# !python -m spacy download en_core_web_trf

generate base_config.cfg and base_config_gpu.cfg on https://spacy.io/usage/training#quickstart

In [3]:
# !python -m spacy init fill-config base_config.cfg config.cfg
# !python -m spacy init fill-config base_config_gpu.cfg config_gpu.cfg

## Gather Data

###  Convert Excel to Txt
upload the excel file to http://15.15.166.35:18888/tree/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx

~~~shell
!pip install xlrd==1.2.0
~~~

In [4]:
# !ls -l /tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx/ 

In [5]:
# ! /tf/eipi10/jian-xu3/snps-classification/snps/etl/xlsx2txt.py  case_note_ner '' 1 '\t'

### Load Txt File

In [6]:

def load_data(text_file):
    names = ['Case_ID', 'Work_Order_Number', 'Created_On', 'Service_Account', 'Due_Date', 
             'Action_Booking_status', 'Action_Date', 'Currently_Worked_by', 'Owner', 'Product_Line', 
             'Follow_Up_Reason_Code', 'Response_Time', 'Work_Order_Type', 'Postal_Code', 'Active_Booking', 
             'Follow_Up_Type', 'Business_Segment', 'Follow_Up_Note', 'Reschedule_Note']
    dtypes = [np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str,
              np.str, np.str, np.str, np.str]
    dtypes = {name:dtype for name, dtype in zip(names, dtypes)}
    df = pd.read_table(text_file, names=names, dtype=dtypes, quoting=csv.QUOTE_NONE)

    df.index = range(1, len(df)+1) 
    df['id'] = df.index
      
    return df

text_file='/tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/txt/3RS_July_Extraction_with_notes_004.txt'
df_case = load_data(text_file)

In [7]:
print('-'*50)
print(len(df_case))

print('-'*50)
display(df_case.head(5))

print('-'*50)
display(df_case.describe().T)

print('-'*50)
df_case.info()



--------------------------------------------------
7039
--------------------------------------------------


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,Follow_Up_Reason_Code,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4
5,5061278521,WO-013010632-4,2021-06-16 12:17:00,MR. REGINALD JOSEPH,2021-07-15 14:42:00,Reschedule,2021-07-15 13:09:00,NaN,JOCILYN ANDREA SANDI DURAN,Commercial Premium Notebooks,NaN,NCD,Break Fix,20785,USA-PR-USYWWDWN_E_A_EEG,Reschedule,Computing,NaN,NaN,5


--------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
id,7039.0,3520.0,2032.128605,1.0,1760.5,3520.0,5279.5,7039.0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7039 entries, 1 to 7039
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Case_ID                7039 non-null   object
 1   Work_Order_Number      7039 non-null   object
 2   Created_On             7039 non-null   object
 3   Service_Account        7038 non-null   object
 4   Due_Date               7017 non-null   object
 5   Action_Booking_status  4223 non-null   object
 6   Action_Date            3408 non-null   object
 7   Currently_Worked_by    887 non-null    object
 8   Owner                  7039 non-null   object
 9   Product_Line           6457 non-null   object
 10  Follow_Up_Reason_Code  2864 non-null   object
 11  Response_Time          7039 non-null   object
 12  Work_Order_Type        7039 non-null   object
 13  Postal_Code            7039 non-null   object
 14  Active_Booking       

In [8]:
print(df_case.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1666
Onsite Stock Replenishment     376
Partner Request Reassign        16
Request Reassign              2100
Reschedule                     434
Revisit                       2447
dtype: int64


### Data Cleaning

In [9]:
def filter_out_note_na(df):
    df_filter = df.loc[(df.Follow_Up_Note.notna()) | (df.Reschedule_Note.notna())].copy()
    return df_filter

df_train_test = filter_out_note_na(df_case)
print(len(df_train_test))

# df_train_test['notes'] = [('' if pd.isna(fun) else fun)  + ('' if pd.isna(crn) else ' | ' + crn)
#                           for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test['notes'] = [(crn if pd.isna(fun) else (fun if pd.isna(crn) else fun + '.' + crn))
                          for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test

6110


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1,#NORMAL Part escalated
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2,Please cancel backordered part. Customer has c...
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3,According to bryan this laptop shows repair co...
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4,Back order part
6,5064053283,WO-013536670-2,2021-06-30 11:06:00,Ford Motor Company of Canada,2021-07-02 08:30:00,Request Reassign,2021-07-05 14:45:00,NaN,Gabriel Bustos,A3 Laserjet,...,NCD,IMACD,N9A,Rick Harbarenko,Request Reassign,dMPS,NaN,Please send this case to Corey Harbarenko. Cor...,6,Please send this case to Corey Harbarenko. Cor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,5069292833,WO-014560602,2021-07-16 16:23:00,Kaiser Foundation Health Plan Inc.,2021-07-19 15:09:00,Additional Parts Required,2021-07-16 16:00:00,NaN,6 CSPAE166,A4 PageWide SMB & Enterprise managed,...,Not available,Break Fix,91786,Anthony Garcia,Additional Parts Required,pMPS,NaN,Please order the following part. Qty. 1 B5L04...,7035,Please order the following part. Qty. 1 B5L04...
7036,5069293232,WO-014560595,2021-07-16 16:21:00,CDW - St. Lukes Hospital (Government),2021-07-20 16:26:00,Request Reassign,NaN,NaN,1 CSPAE131,A4 Laserjet SMB,...,Not available,Break Fix,55805,Brock Tharp,Request Reassign,pMPS,NaN,Please reassign to lms,7036,Please reassign to lms
7037,5069293974,WO-014560699,2021-07-16 17:06:00,LITHIA NISSAN OF FRESNO,2021-07-19 18:00:00,Request Reassign,2021-07-19 13:00:00,Aaron Stavinsky,Luis Alejandro Vargas,A4 Laserjet Enterprise,...,NCD,Preventative Maintenance,93710,IS-Aaron Stavinsky,Request Reassign,Printing,NaN,dahlia.garcia@hp.com,7037,dahlia.garcia@hp.com
7038,5069294340,WO-014560697,2021-07-16 17:04:00,"Best Buy co., Inc.",2021-07-20 18:00:00,NaN,NaN,NaN,Subir Das,Retail Solutions Core,...,NCD,Break Fix,95678,PR-David Popadiuc,Revisit,Computing,Please order 842275-001 and have it ship to HF...,NaN,7038,Please order 842275-001 and have it ship to HF...


In [10]:
print(df_train_test.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1579
Onsite Stock Replenishment     375
Partner Request Reassign        13
Request Reassign              1737
Reschedule                     136
Revisit                       2270
dtype: int64


### Generate NER Label

In [11]:
def standardize_text1(text):
    text = text.strip().lower()
    # add space between punctuation
    text = re.sub(r'([.\\!?,\'()\[\]"|;])', r' \1 ', text)
#     # remove characters except for English letters and some punctuations
#     text = re.sub(r"[^A-Za-z\.\-\?\!\,\#\@\% ]", "", text)
    # remove extra spaces
    text = re.sub(r'[" "]+', " ", text)
    return text

In [12]:
nlp = spacy.load("en_core_web_sm")
notes = [standardize_text1(note) for note in df_train_test.notes]
docs = nlp.pipe(notes)

In [13]:
def get_samples(df):
    df_filter = df.loc[(df.Work_Order_Number=='WO-013865844-1') | 
                       (df.Work_Order_Number=='WO-013910857') |
                       (df.Work_Order_Number=='WO-014466680') |
                       (df.Work_Order_Number=='WO-014466875') |
                       (df.Work_Order_Number=='WO-014466875-1') 
                      ].copy()
    return df_filter

df_sample = get_samples(df_train_test)
df_sample

,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
21,5065852054,WO-013865844-1,2021-07-02 14:10:00,KAISER PERMANENTE,2021-07-16 18:00:00,NaN,NaN,Ryan Bartoli,Ezequiel Jara Corao,Detachables,...,NCD,Break Fix,94553,CT-Ryan Bartoli,Revisit,Computing,Please create a revisit for Wednesday,#NORMAL Please order part # L51580-001 and L31...,21,Please create a revisit for Wednesday.#NORMAL ...
29,5066093878,WO-013910857,2021-05-14 17:16:00,AMD,2021-07-02 17:00:00,NaN,NaN,NaN,JORGE DANIEL BENAVIDEZ AVILA,Commercial Premium Notebooks,...,NCD,Break Fix,78735,PR-Rob Silcocks,Revisit,Computing,Please order battery 933321-855 ship to UPS H...,Please reassign to Rob Silcocks,29,Please order battery 933321-855 ship to UPS H...
2978,5068832635,WO-014466680,2021-07-08 10:24:00,Jabil circuit Inc,2021-07-09 17:00:00,Additional Parts Required,2021-07-15 08:00:00,NaN,7 CSPAE147,Large Format Design Pagewide XL HW,...,NCD,Break Fix,87106,PR-Isaac Lucero,Additional Parts Required,dMPS,NaN,"Per L2 please order: 2YB64-67005, 2YB64-67003 ...",2978,"Per L2 please order: 2YB64-67005, 2YB64-67003 ..."
3025,5068836700,WO-014466875,2021-07-08 10:49:00,Border Patrol and Customs,2021-07-14 16:00:00,NaN,NaN,NaN,Abu Ambia,Large Format Design HW,...,NCD,Break Fix,79925,PR-Manuel Bailon,Revisit,Printing,Please create a revisite for 7-16-2021 at 1300...,NaN,3025,Please create a revisite for 7-16-2021 at 1300...
3026,5068836700,WO-014466875-1,2021-07-14 15:01:00,Border Patrol and Customs,2021-07-16 16:00:00,NaN,NaN,NaN,Lisbeth Alejandra Sibaja,Large Format Design HW,...,NCD,Break Fix,79925,PR-Manuel Bailon,Revisit,Printing,Please create a revisite for 7-23-2021 for pri...,NaN,3026,Please create a revisite for 7-23-2021 for pri...


In [14]:

sample_notes = [standardize_text1(note) for note in df_sample.notes]
sampel_docs = nlp.pipe(sample_notes)
for i, doc in enumerate(sampel_docs):
    print('-'*50)
    print(doc.text)
    print([(token.text, token.pos_) for token in doc])
    print([(ent.text, ent.label_) for ent in doc.ents]) 

--------------------------------------------------
please create a revisit for wednesday . #normal please order part # l51580-001 and l31365-001 to concord , california hal setting up time to meet with customer . 
[('please', 'INTJ'), ('create', 'VERB'), ('a', 'DET'), ('revisit', 'NOUN'), ('for', 'ADP'), ('wednesday', 'PROPN'), ('.', 'PUNCT'), ('#', 'NOUN'), ('normal', 'ADJ'), ('please', 'INTJ'), ('order', 'VERB'), ('part', 'NOUN'), ('#', 'SYM'), ('l51580', 'PROPN'), ('-', 'PUNCT'), ('001', 'NUM'), ('and', 'CCONJ'), ('l31365', 'NOUN'), ('-', 'PUNCT'), ('001', 'NUM'), ('to', 'ADP'), ('concord', 'PROPN'), (',', 'PUNCT'), ('california', 'PROPN'), ('hal', 'NOUN'), ('setting', 'VERB'), ('up', 'ADP'), ('time', 'NOUN'), ('to', 'PART'), ('meet', 'VERB'), ('with', 'ADP'), ('customer', 'NOUN'), ('.', 'PUNCT')]
[('wednesday', 'DATE'), ('# l51580-001', 'MONEY'), ('concord', 'GPE'), ('california', 'GPE')]
--------------------------------------------------
please order battery 933321-855 ship to ups

In [15]:
patterns = [
 {'expression': r'[a-zA-Z]\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d-\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]-\d\d\d\d-\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d[a-zA-Z]-\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]66-01190A', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d[a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d-\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z]\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
#  {'expression': r'order', 'label': 'ACTION'}, 
#  {'expression': r'recommended', 'label': 'ACTION'}, 
#  {'expression': r'hfpu', 'label': 'ACTION'}, 
#  {'expression': r'ship', 'label': 'ACTION'}, 
#  {'expression': r'fedex', 'label': 'ACTION'}, 
#  {'expression': r'hal', 'label': 'ACTION'}, 
#  {'expression': r'site', 'label': 'ACTION'}, 
#  {'expression': r'send', 'label': 'ACTION'}, 
#  {'expression': r'location', 'label': 'ACTION'}, 
#  {'expression': r'pickup', 'label': 'ACTION'}, 
#  {'expression': r'sent', 'label': 'ACTION'}, 
#  {'expression': r'requested', 'label': 'ACTION'}, 
#  {'expression': r'reorder', 'label': 'ACTION'}, 
#  {'expression': r'street', 'label': 'ACTION'}, 
#  {'expression': r'hfp', 'label': 'ACTION'}, 
#  {'expression': r'shipped', 'label': 'ACTION'}, 
#  {'expression': r'delivery', 'label': 'ACTION'}, 
#  {'expression': r'recommend', 'label': 'ACTION'}, 
#  {'expression': r'warehouse', 'label': 'ACTION'}    
]

In [17]:
common_words_patterns = [
 {'expression': r'order', 'label': 'ACTION'}, 
 {'expression': r'recommended', 'label': 'ACTION'}, 
 {'expression': r'hfpu', 'label': 'ACTION'}, 
 {'expression': r'ship', 'label': 'ACTION'}, 
 {'expression': r'fedex', 'label': 'ACTION'}, 
 {'expression': r'hal', 'label': 'ACTION'}, 
 {'expression': r'site', 'label': 'ACTION'}, 
 {'expression': r'send', 'label': 'ACTION'}, 
 {'expression': r'location', 'label': 'ACTION'}, 
 {'expression': r'pickup', 'label': 'ACTION'}, 
 {'expression': r'sent', 'label': 'ACTION'}, 
 {'expression': r'requested', 'label': 'ACTION'}, 
 {'expression': r'reorder', 'label': 'ACTION'}, 
 {'expression': r'street', 'label': 'ACTION'}, 
 {'expression': r'hfp', 'label': 'ACTION'}, 
 {'expression': r'shipped', 'label': 'ACTION'}, 
 {'expression': r'delivery', 'label': 'ACTION'}, 
 {'expression': r'recommend', 'label': 'ACTION'}, 
 {'expression': r'warehouse', 'label': 'ACTION'}   
]

In [18]:
def generate_label(texts, patterns={}):   
    
    def entity_from_patterns(text, patterns, pattern_matchs):        
        entities = {}
#         print('-'*50)  
        for pattern in patterns:
            entities = entity_from_pattern(text, pattern, entities, pattern_matchs)
        entities = [value for _, value in sorted(entities.items(), key=lambda item: item[0][0])]
        return entities

    def entity_from_pattern(text, pattern, entities, pattern_matchs):
        expression = pattern['expression']
        label = pattern['label']
        for match in re.finditer(expression, text):
            start, end = match.span()    
            add_key = True
            if start-1 >= 0 and text[start-1]!=' ':
                add_key = False
            if end < len(text) and text[end]!=' ':
                add_key = False                
                
#             drop_keys = []
#             for key, value in entities.items():
#                 start_, end_ = key
#                 if start<start_ and end>end_:
#                     drop_keys.append(key)
#                 if start>start_ and end<end_:
#                     add_key = False
#             for key in drop_keys:
#                 entities.pop(key)

            if add_key: 
#                 print(text, expression)
                pattern_matchs[expression] = pattern_matchs[expression] + 1
                entities[(start, end)] = (start, end, label) 
        return entities
        
    examples = []
    pattern_matchs = {pattern['expression']:0 for pattern in patterns}
    for text in texts:
        entities = entity_from_patterns(text, patterns, pattern_matchs)        
        example = (text, {"entities": entities})
        examples.append(example)
    pattern_matchs = [(key, value) for key, value in sorted(pattern_matchs.items(), key=lambda item: -item[1]) if value>0]
#     pattern_matchs = [(key, value) for key, value in pattern_matchs if value>0]
        
    return examples, pattern_matchs

see some sample data.

In [19]:
def show_entity_label(text_entities, pattern_matchs):
    for example in text_entities:
        print('-'*50)
        text, annotations = example
        print(text) 
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')
    print('='*50)
    print(*pattern_matchs, sep='\n')
    
texts = [standardize_text1(note) for note in df_sample.notes]  
sample_text_entities, sample_pattern_matchs = generate_label(texts, patterns=patterns)
show_entity_label(sample_text_entities, sample_pattern_matchs)

--------------------------------------------------
please create a revisit for wednesday . #normal please order part # l51580-001 and l31365-001 to concord , california hal setting up time to meet with customer . 
('l51580-001', 68, 78, 'PART_NUM')
('l31365-001', 83, 93, 'PART_NUM')
--------------------------------------------------
please order battery 933321-855 ship to ups hold for pickup , tuscany way , austin . . please reassign to rob silcocks
('933321-855', 21, 31, 'PART_NUM')
--------------------------------------------------
per l2 please order: 2yb64-67005 , 2yb64-67003 xl3600 , 6kd23-67021 xl3600 , 6kd23-67030 , and two 2yb64-67002
('2yb64-67005', 21, 32, 'PART_NUM')
('2yb64-67003', 35, 46, 'PART_NUM')
('6kd23-67021', 56, 67, 'PART_NUM')
('6kd23-67030', 77, 88, 'PART_NUM')
('2yb64-67002', 99, 110, 'PART_NUM')
--------------------------------------------------
please create a revisite for 7-16-2021 at 1300 order part number q5669-60687 please order 2 q6683-67001 t8w18-67001 s

In [56]:
texts = [standardize_text1(note) for note in df_train_test.notes]           
train_texts, test_texts = train_test_split(texts, test_size=0.25, random_state=199)
print(len(train_texts))
print(len(test_texts))

4582
1528


In [67]:
text_entities, pattern_matchs = generate_label(texts, patterns=patterns)
tran_text_entities, train_pattern_matchs = generate_label(train_texts, patterns=patterns)
test_text_entities, test_pattern_matchs = generate_label(test_texts, patterns=patterns)

In [68]:
print(len(text_entities))
print(*pattern_matchs, sep='\n')

6110
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 1449)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 1089)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 643)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d\\d\\d\\d', 597)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 547)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 400)
('[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 271)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 166)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 122)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z]\\d\\d', 46)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 39)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]', 21)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 14)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 6)
('\\d\\d-\\d\\d\\d\\d\\d-\\d\\d\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z][a-zA-Z]\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z

In [66]:
print(len(train_pattern_matchs))
print(*train_pattern_matchs, sep='\n')  

23
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 987)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 825)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 489)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d\\d\\d\\d', 450)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 414)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 287)
('[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 201)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 116)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 97)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z]\\d\\d', 34)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 31)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]', 14)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 12)
('\\d\\d-\\d\\d\\d\\d\\d-\\d\\d\\d', 5)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 4)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z][a-zA-Z]\\d', 3)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 3)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z]\\d\

In [59]:
print(len(test_pattern_matchs))
print(*test_pattern_matchs, sep='\n')  

17
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 462)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 264)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 154)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d\\d\\d\\d', 147)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 133)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 113)
('[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 70)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 50)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 25)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z]\\d\\d', 12)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 8)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]', 7)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 2)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z][a-zA-Z]\\d', 2)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 2)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 2)
('[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d', 1)


In [60]:
# train_data, test_data = train_test_split(text_entities, test_size=0.25, random_state=199)

# print(len(train_data))
# print(len(test_data))

### Data Augmentation

In [61]:
exrex.getone(r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d')

'H68124-P71'

In [74]:
def get_augument_rates(pattern_counts):
    pattern_counts = np.array([300+np.sqrt(count-300) if count>300 else count for count in pattern_counts ] )
    pattern_counts = max(pattern_counts) - pattern_counts
    augument_rates = pattern_counts/np.sum(pattern_counts)
    
    return augument_rates

pattern_matchs_dict = {key:value for key, value in pattern_matchs}   
part_num_patterns = [pattern for pattern in patterns if pattern['label']=='PART_NUM' ]
part_num_pattern_counts = [pattern_matchs_dict.get(pattern['expression'], 0) for pattern in part_num_patterns]

# part_num_augument_rates = get_augument_rates(part_num_pattern_counts) 
# print(max(part_num_augument_rates), sum(part_num_augument_rates))
# [(cnt, rate) for cnt, rate in zip(part_num_pattern_counts, part_num_augument_rates)]

0.006985745663797591 1.000000000000002


[(547, 0.0009338009370212769),
 (400, 0.00123351925408684),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (6, 0.0051183136013677275),
 (0, 0.006985745663797591),
 (2, 0.005931440143558504),
 (643, 0.0007788876662032042),
 (166, 0.00207415930650118),
 (0, 0.006985745663797591),
 (39, 0.0034456378276801676),
 (1089, 0.00027387631733373285),
 (0, 0.006985745663797591),
 (122, 0.002367635541956403),
 (6, 0.0051183136013677275),
 (0, 0.006985745663797591),
 (1449, 0.0),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (5, 0.00526624742148503),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745663797591),
 (0, 0.006985745

In [77]:
a = {i:cnt for i, cnt in enumerate(np.random.randint(10, size=10)) if cnt<5}  
a 

{6: 3, 7: 1, 8: 4, 9: 3}

In [28]:
def augment(data, patterns, pattern_matchs, augument_rates, min_count=100, augument_num=40000):
    i = 0
    new_texts = []
    
    pattern_matchs_dict = {key:value for key, value in pattern_matchs}   
    part_num_patterns = [pattern for pattern in patterns if pattern['label']=='PART_NUM' ]
    part_num_pattern_counts = [pattern_matchs_dict.get(pattern['expression'], 0) for pattern in part_num_patterns]
    part_num_pattern_counts = {int(101-math.log(count+1)) 
                               for index:count in enumeraten(part_num_pattern_counts) if count<100}
    
    while i<augument_num and len(part_num_pattern_counts)>0:
        data_index = int(np.random.randint(len(data), size=1))
        text, annotations = data[data_index]
        entities = [(start, end, label) for start, end, label in annotations['entities'] if label=='PART_NUM']        
        if len(entities)<=0:
            continue
        
        entities.reverse()
        new_text = text
        for start, end, label in entities:                       
            pattern_index, count = random.choice(list(part_num_pattern_counts.items()))
            pattern = patterns[pattern_index]['expression']
            part_num = exrex.getone(pattern).lower()
            new_text = new_text[0:start] + part_num + new_text[end:] 
            part_num_pattern_counts[pattern_index] = count-1
            if part_num_pattern_counts[pattern_index] <= 0:
                part_num_pattern_counts.pop()

            
            
        new_texts.append(new_text)   

        i = i + 1
        if i % 5000==0:
            print(f'{i}/{augument_num}')
    
    while i<augument_num:
        data_index = int(np.random.randint(len(data), size=1))
        text, annotations = data[data_index]
        entities = [(start, end, label) for start, end, label in annotations['entities'] if label=='PART_NUM']        
        if len(entities)<=0:
            continue
        entities.reverse()
        new_text = text
        for start, end, label in entities:                       
            pattern_index = int(np.random.choice(a=len(augument_rates), size=1, replace=True, p=augument_rates))
            pattern = patterns[pattern_index]['expression']
            part_num = exrex.getone(pattern).lower()
            new_text = new_text[0:start] + part_num + new_text[end:] 
        new_texts.append(new_text)   
#         print('-'*50)
#         print(text)
#         print(new_text)
#         print([text[start:end] for start, end, label in entities])
        i = i + 1
        if i % 5000==0:
            print(f'{i}/{augument_num}')
            
    return new_texts
        
    
sample_augment_texts =  augment(train_data[0:10], patterns, part_num_augument_rates, augument_num=10)
sample_augment_texts

['please order part number wzo-831qv and part number kih-cki2 and part number 9h-oc299-i8 to hal for pickup in west valley utah . ',
 'please order part number mio-egb847p and part number fuvcnj-h6424kic and part number bnt-94b9407-tj to hal for pickup in west valley utah . ',
 'please order nbd and ship to fedex service center at 16815 redmond way suite 220 , redmond , wa 98052 required parts: rm1-5555-010cn\xa0 \xa0\xa0 \xa0rm1-5777-000cn\xa0 rk2-2415-000cn\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 rm1-5521-010cn\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 cc493-67922\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 qma-86q7ox1',
 'please create a revisit , and transfer parts to the revisit . also please order the pn : ajzyrwm-8511ta97 and hfpu',
 'please create a revisit , and transfer parts to the revisit . also please order the pn : ysoliqr-3059yir6 and hfpu',
 'please create a revisit , and transfer parts to the revisit . also please order the pn : soj-2g928 and hf

In [29]:
patterns

[{'expression': '[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 'label': 'PART_NUM'},
 {'expression': '[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d\\d\\d-\\d[a-zA-Z]\\d', 'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]',
  'label': 'PART_NUM'},
 {'expression': '[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d',
  'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 'label': 'PART_NUM'},
 {'expression': '[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d',
  'label': 'PART_NUM'},
 {'expression': '[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]',
  'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d-\\d\\d\\d\\d', 'label': 'PART_NUM'},
 {'expression': '\\d\\d\\d\\d-\\d\\d\\d\\d-\\d', 'label': 'PART_NUM'},
 {'expression': '\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\

In [30]:
augment_texts =  augment(train_data, patterns, part_num_augument_rates, augument_num=30000)
print(len(augment_texts))

5000/30000
10000/30000
15000/30000
20000/30000
25000/30000
30000/30000
30000


In [31]:
augment_text_entities, augment_pattern_matchs = generate_label(augment_texts, patterns=patterns)

In [32]:
augment_pattern_matchs

[('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d-\\d\\d\\d\\d', 2694),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d\\d\\d', 1278),
 ('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]', 1264),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d\\d[a-zA-Z][a-zA-Z]', 1260),
 ('\\d\\d\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z]', 1254),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z]\\d\\d\\d\\d\\d', 1240),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 1234),
 ('[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d\\d\\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]',
  1234),
 ('[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d\\d[a-zA-Z][a-zA-Z][a-zA-Z]\\d',
  1228),
 ('[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d',
  1210),
 ('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 1206),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d\\d\\d', 1176),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d\

In [33]:
train_data = train_data + augment_text_entities
print(len(train_data))

34582


### Generate NER data

In [34]:
def generate_data(data):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return db
    
train_spacy = generate_data(train_data)
train_spacy.to_disk("./train_aug.spacy") # save the docbin object
test_spacy = generate_data(test_data) 
test_spacy.to_disk("./test_aug.spacy") # save the docbin object

100%|██████████| 1528/1528 [00:01<00:00, 1191.94it/s]


In [35]:
!ls -l . 

total 7868
-rw-r--r--. 1 root root    1916 Aug 23 01:30 base_config.cfg
-rw-r--r--. 1 root root    1664 Aug 23 09:48 base_config_gpu.cfg
-rw-r--r--. 1 root root    2833 Aug 25 00:26 config.cfg
-rw-r--r--. 1 root root    2625 Aug 25 00:11 config_gpu.cfg
-rw-r--r--. 1 root root  289100 Aug 25 00:31 custom_ner.ipynb
-rw-r--r--. 1 root root  163582 Aug 25 00:33 custom_ner_aug.ipynb
-rw-r--r--. 1 root root  142918 Aug 25 00:29 custom_ner_cpu.ipynb
drwxr-xr-x. 5 root root      43 Aug 24 09:04 output
-rw-r--r--. 1 root root  227413 Aug 25 00:27 test.spacy
-rw-r--r--. 1 root root  229957 Aug 25 00:33 test_aug.spacy
-rw-r--r--. 1 root root  660076 Aug 25 00:27 train.spacy
-rw-r--r--. 1 root root 6312285 Aug 25 00:33 train_aug.spacy


## Train the model

In [36]:
# !python -m spacy train config.cfg --output ./output/cpu --paths.train ./train.spacy --paths.dev ./test.spacy

In [39]:
!python -m spacy train config_gpu.cfg --output ./output/gpu_aug --paths.train ./train_aug.spacy --paths.dev ./test_aug.spacy --gpu-id 0

2021-08-25 00:34:45.007545: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-25 00:34:48,848] [INFO] Set up nlp object from config
[2021-08-25 00:34:48,874] [INFO] Pipeline: ['transformer', 'ner']
[2021-08-25 00:34:48,885] [INFO] Created vocabulary
[2021-08-25 00:34:48,887] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

## Evaluate

###  Check test data

In [40]:
nlp = spacy.load("output/gpu_aug/model-last") 

In [41]:
options = {'colors': {'ACTION':"#56D7C4", 'PART_NUM':"#92E0AA"} }

def show_text(text, annotations=None, options=options, show_detail=True, standardize_text=None):
    if standardize_text is not None:
        text = standardize_text(text)
    doc = nlp(text)
    print('='*100)
    if show_detail:
        print(doc.text) 
    spacy.displacy.render(doc, style='ent', options=options)
    if show_detail and annotations is not None and len(annotations['entities'])>0:
        print('-'*25, 'acutal entities', '-'*25)
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')   
    if show_detail and len(doc.ents)>0:
        print('-'*25, 'predict entities', '-'*25)
        print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n')      

def show_texts(data, indexes=None, options=options, show_detail=True, standardize_text=None):
    def show_text_(data):
        if isinstance(data, str):
            show_text(data, None, options=options, show_detail=show_detail, standardize_text=standardize_text)
        else:
            text, annotations = data
            show_text(text, annotations, options=options, show_detail=show_detail, standardize_text=standardize_text)  
            
    if indexes is not None:
        for i in indexes:
            show_text_(data[i])   
    else:
        for one_data in data: 
            show_text_(one_data)      
    
                  

In [42]:
indexes = np.random.randint(0, len(test_data), 10)
show_texts(test_data, indexes) 

#normal


/usr/local/lib/python3.6/dist-packages/spacy/displacy/__init__.py:191: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


please route to jamaul harris . cu not in office i called chris on7/19/21 and left a message . i am awaiting a response . jh


please send to doug houseknecht please order reccomended part overnight to fedex manitou rd attention kevin deasey rm2-5752-000cn #normal


------------------------- acutal entities -------------------------
('rm2-5752-000cn', 115, 129, 'PART_NUM')
------------------------- predict entities -------------------------
('rm2-5752-000cn', 115, 129, 'PART_NUM')
please cancel escalated part . . r-r case to ce dwayne martin spoke with customer , will go to his location friday . 


...

------------------------- acutal entities -------------------------
('714657-001', 19, 29, 'PART_NUM')
------------------------- predict entities -------------------------
('714657-001', 19, 29, 'PART_NUM')
please create revisit , i ' m waiting for call from l2 , l3


please part number 910584-001 and ship next business day to my hfpu site at 155 wellington street west , toronto


------------------------- acutal entities -------------------------
('910584-001', 19, 29, 'PART_NUM')
------------------------- predict entities -------------------------
('910584-001', 19, 29, 'PART_NUM')
loaner in place further diagnostics required nh


#normal assign 7/16/21 emailed have part emailed repair request . 7/20/21 confirmed appt 👍 . 


------------------------- predict entities -------------------------
('👍 .', 89, 92, 'PART_NUM')
please order recommended part nbd to my hold for pickup


In [43]:
show_texts(test_data, indexes, show_detail=False) 

### Check Error Data

In [44]:
def get_error_data(data):
    error_data = []
    for i in range(len(data)):
        text, annotations = data[i]
        doc = nlp(text)
        entities = annotations['entities']
        if len(entities) != len(doc.ents):
            error_data.append(data[i])
            print(i, len(error_data))
        else:
            exit_flag = False
            actual_entities = {(start, end, label): label for start, end, label in entities}
            predict_entities = {(ent.start_char, ent.end_char, ent.label_): ent.label_ 
                                for ent in doc.ents}
            
            for key, _ in actual_entities.items():                
                if key not in predict_entities:
                    print(actual_entities, predict_entities, sep='\n') 
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
            if exit_flag: continue
            for key, _ in predict_entities.items():
                if key not in actual_entities:
                    print(actual_entities, predict_entities)
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
                
    return error_data

error_data = get_error_data(test_data)

34 1
73 2
89 3
131 4
168 5
169 6
229 7
277 8
281 9
284 10
341 11
382 12
408 13
423 14
438 15
462 16
498 17
548 18
561 19
573 20
599 21
607 22
610 23
628 24
670 25
691 26
721 27
785 28
834 29
859 30
891 31
948 32
1002 33
{(45, 56, 'PART_NUM'): 'PART_NUM'}
{(32, 42, 'PART_NUM'): 'PART_NUM'}
1006 34
{(45, 56, 'PART_NUM'): 'PART_NUM'} {(32, 42, 'PART_NUM'): 'PART_NUM'}
1006 35
1046 36
1163 37
1227 38
1240 39
1250 40
1263 41
1293 42
1303 43
1330 44
1360 45
1366 46
1404 47
1415 48
1434 49
1444 50
1456 51
1471 52
1473 53
1526 54


In [48]:
print(len(error_data))

54


In [49]:
show_texts(error_data)  

i called to end user by the phone i got from the tickets and by her cells phone . 425-985-2765 . unable to reach to end user leaved voice messages . i went to the site end user doesn ' t answer phone call and leave voice messages . 07/14/2021 . 


------------------------- predict entities -------------------------
('425-985-2765', 82, 94, 'PART_NUM')
please create a clone for the revisit for july 19 from 1300-1400 . on break- site closed before i could complete


------------------------- acutal entities -------------------------
('1300-1400', 55, 64, 'PART_NUM')
revisit 7/9 please order dell-snpcrxj6c to san diego hal . please refer part order to multivendor team - dell diagnosis - failed memory diagnostics . waiting for user , but never returned call back . 


------------------------- predict entities -------------------------
('dell-snpcrxj6c', 25, 39, 'PART_NUM')
please create a revisit schedule on 7/16 and order part 3gy31-67901 / 5851-6712 qty 1 send to hal 83003714


------------------------- acutal entities -------------------------
('3gy31-67901', 56, 67, 'PART_NUM')
('5851-6712', 70, 79, 'PART_NUM')
------------------------- predict entities -------------------------
('5851-6712', 70, 79, 'PART_NUM')
please reorder cable kit to be shipped nbd to bloomington il hal m15627-001 7/21 customer would like to reschedule the call on monday . please add this part to be shipped nbd to bloomington il m15627-001


...

------------------------- acutal entities -------------------------
('j8a10-60113', 18, 29, 'PART_NUM')
('b5l47-67018', 89, 100, 'PART_NUM')
------------------------- predict entities -------------------------
('b5l47-67018', 89, 100, 'PART_NUM')
please order lifter drive assembly rm2-0010-000cn and formatter b5l46-67909 to the bangor fedex hfpu . please move unused parts to the followup . . #normal


------------------------- acutal entities -------------------------
('rm2-0010-000cn', 35, 49, 'PART_NUM')
('b5l46-67909', 64, 75, 'PART_NUM')
------------------------- predict entities -------------------------
('b5l46-67909', 64, 75, 'PART_NUM')
l09562-001 quantity 1 ship to 83004650


------------------------- acutal entities -------------------------
('l09562-001', 0, 10, 'PART_NUM')
please reassign call to john fiorini . ce is aware . rio 1 ( 772 ) 388-4339 cornelius ( 772 ) 388-4386


------------------------- predict entities -------------------------
('388-4339', 67, 75, 'PART_NUM')
('388-4386', 94, 102, 'PART_NUM')
please order recommended part nbd to wo address under user’s name 7/16- part eta change from 10:30am to 4pm part eta : end of the day ! e-mail user with tracking information . waiting for part 7/19- part status: “potentially delayed” 7/20- please order j


------------------------- predict entities -------------------------
('“potentially', 212, 224, 'PART_NUM')
('” 7/20-', 232, 239, 'PART_NUM')


In [47]:
doc = nlp('rm26322-000cn PART_NUM e6b69-67901 PART_NUM rm1-8413-000cn PART_NUM please send ACTION to fedex ACTION 3242 herrman rd garland , texas 75041 on 2021-08-21')
print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n') 
spacy.displacy.render(doc, style='ent', options=options)

('e6b69-67901', 23, 34, 'PART_NUM')
('rm1-8413-000cn', 44, 58, 'PART_NUM')


### Some Features and Issues

#### Miss some part-numbers 

In [50]:
sample_parts = ['M14302-161', '926537-001', 'M46915-D01', '140314-FB2', '176253-8B0', 
               '442010-ABC', 'L07896-DB1', '168757-B21', 'FE-15520-01', 'RM1-7867-000CN', 
               '5066-4719', '5066-3872-1', '3HZ90-30001', 'T6M22-30001', '98544-04105', 
               'C8109-69018', 'L26480-DB1', '632427-001B', 'CN463-69003', 'RM1-1298BULK', 
               '8-752-078-46', 'X-4035-119-1', 'TBLB3002-A04', '4822-701-15319', '08-2212C-33W', 
               '19-40065-011', 'ATT09850-66532', '98564-66572-SQ', '897-250144AA', '738-576-94', 
               '1FQ6-0001', '1818-7469-LGSA', '1820-6361M', '1818-6868-SAM', '3138-107-96160', 
               '686858R-999', 'MD30C-A2', '29-26196-00', 'DEL-7D092', 'LEX-40X2384', 'LEX-40X2831H', 
               'SAM-6107001172', 'SHA-1625DS51', 'XER-006R01275', 'BRO-TN115BK', 'DEL-310-5811', 
               'DEL-PK496', 'KYO-1702F97US0', 'LA95-CA', 'KYO-DK-310H', 'EPS-C31CD38A9921', 
               'KYO-F994091H', 'EPS-CEPS-003G', 'LEX-24015SA', 'LEX-62D1X00', 'LEX-72K0DV0', 
               'LEX-801HC', 'LEX-C5220CSH', 'TRO-78-24619-001', 'LEX-C544X1CG', 'HPE-749797-001', 
               'SFT-P-895', 'LEX-E260A21A-C', 'RIC-D0296509', 'SEA-ST1000LM049', 'LEX-12A8400-PC', 
               'HPE-JL258A', 'RIC-400507', 'FUJ-FPCPR362AQ', 'TOS-6LE8296100', 'XER-003K04980', 
               'XER-116111500', 'ATS-2711FXSC-901', 'BRO-TN336BK', 'DEL-310-5807', 'HPI-CF258A-M', 
               'LEX-T101-0000000', 'DEL-9PN5P', 'SAM-BA92-08880A', 'APC-RBC7', 'KMH-4448-121', 
               'SXC-C7ABTTAAB', 'TRO-04621201', 'VEF-28924-04-R', 'WFP-HPM527Z', 'KYO-1702LK0UN2', 
               'APC-SUA1500RM2U', 'KYO-302NM18021', 'HPE-JH329-61001', 'KYO-1T02GA0US0', 'LEB-04X4674',
               'KYO-302GR93034', 'DEL-V1RX3', 'LEX-40X0593', 'TOS-6LH3460800', 'RIC-AE020171', 
               'XER-59K60140', 'BRO-LM5140001', 'KMH-A00JA56600', 'KYO-302F906240', 'BRO-LEM084001', 
               'XER-116-2035', 'BRO-BU100CLH', '3X-LK465-A2', '4G1-3999-030CN', 'H3980-60002BULK', 
               'SHA-VHI0MFP000', 'TOS-C017839000', 'JC66-01190A', 'XER-3335DNI', 'XER-3635MFP-S', 
               'XER-4622DN', 'XER-7750-Z2', 'DEL-D1320-W3', 'DEL-P637D', 'LEX-C540X35G', 
               'SHA-CPLTM8190F', 'SHACFRM-1380DS53', 'XER-401-0855-0', '90-0077', 'BRO-LF6710001', 
               'Q3938-68001-PCA', 'C314X+49A-90002', '20ER-84530KC', 'RM2-2903-BULK', 'CE506-67919-BU', 
               'CH971-91596', 'DEL-FM235', 'JC44-00210E', 'SHACFRM-1509DS52', 'SHADHAI-5043FCPZ', 
               'SHAGCAB-1102FCB1', '000-50-02-008', '504000226-DD', 'SHAVHPLG217L5A-1', 'SHAVHPOSH03Y02-1', 
               'SHADUNT-8962FCP1', 'SHAKI-OK0010FCPZ', 'SHAQSW-P0506FCZZ', 'SHAVHPGP1A73AR-1', 'SHARH-IX0055FCPZ', 
               '358-000001', 'SHAVHPGP1A71L3-18', 'B23-0994', '504000226-D', '5066-0731', 
               'E0-CABLE-01', 'RM1-8508-010-BU', 'B3M77-67902-BU', 'CA02626-E029FJ', 'CF065-67901-BU', 
               'C2H57-67901-BULK', 'LN08-A2', '693709-001B', 'SS467-67001', 'LEX-70C0D20', 
               'SHAQSW-M0518FCPZ', 'SHALX-BZ0994FCPZ', 'LN08X-TF', 'SHAGCAB-1507FCAZ']

sample_parts = [part.lower() for part in sample_parts]

In [51]:
def get_texts(parts, standardize_text=standardize_text1):
    texts = [standardize_text(f'Please order qty 1 {part} NBD to my HAL in Lenexa. Thank you.') for part in parts]
    return texts

def check_sample_parts(parts):  
    texts = get_texts(parts)
    docs = nlp.pipe(texts)
    texts = []
    error_texts = []
    errpr_parts = []
    for doc, part in zip(docs, parts):
        texts.append(doc.text)
        if len(doc.ents) !=1 or doc.ents[0].text != part.lower():
            error_texts.append(doc.text)
            errpr_parts.append(part)
    return texts, error_texts, errpr_parts
        
parts = ['000-50-02-008']
texts, error_texts, errpr_parts = check_sample_parts(parts)
show_texts(texts, show_detail=True) 

please order qty 1 000-50-02-008 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('000-50-02-008', 19, 32, 'PART_NUM')


In [53]:
parts = sample_parts
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')
show_texts(texts, show_detail=True)  

find 0/160 errors
please order qty 1 m14302-161 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('m14302-161', 19, 29, 'PART_NUM')
please order qty 1 926537-001 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('926537-001', 19, 29, 'PART_NUM')
please order qty 1 m46915-d01 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('m46915-d01', 19, 29, 'PART_NUM')
please order qty 1 140314-fb2 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('140314-fb2', 19, 29, 'PART_NUM')
please order qty 1 176253-8b0 nbd to my hal in lenexa . thank you . 


...

------------------------- predict entities -------------------------
('lex-70c0d20', 19, 30, 'PART_NUM')
please order qty 1 shaqsw-m0518fcpz nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('shaqsw-m0518fcpz', 19, 35, 'PART_NUM')
please order qty 1 shalx-bz0994fcpz nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('shalx-bz0994fcpz', 19, 35, 'PART_NUM')
please order qty 1 ln08x-tf nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('ln08x-tf', 19, 27, 'PART_NUM')
please order qty 1 shagcab-1507fcaz nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('shagcab-1507fcaz', 19, 35, 'PART_NUM')


In [ ]:
error_text_entities, error_pattern_matchs = generate_label(error_texts, patterns=patterns)
show_entity_label(error_text_entities, error_pattern_matchs)

I think that the issues are mainly caused by lacking of these parts in training data. 

There are 2 solutions:

1. merge the results of regular expressions and the model.
2. use data augmentation to generate a lot of random part numbers based on the rules.

####  Find potentail part number

In [ ]:
parts = ['FE-15520-01', 'FE-15520-012']
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')



In [ ]:
some_text_entities, some__pattern_matchs = generate_label(texts, patterns=patterns)
show_entity_label(some_text_entities, some__pattern_matchs)


#### find potential actions 

it has some effects, but not good enough.

In [ ]:
texts = ['order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'orders qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'send qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'sends qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ship qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ships qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 2012-02-02',         
        ]

show_texts(texts, show_detail=False) 

## Output Results

In [ ]:
parts = 